In [276]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [277]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" or "tensorflow" or "torch" 

import keras_cv
import keras
import keras.backend as K
import tensorflow as tf
import tensorflow_io as tfio

import numpy as np 
import pandas as pd

from glob import glob
from tqdm import tqdm

import librosa
import IPython.display as ipd
import librosa.display as lid

import matplotlib.pyplot as plt
import matplotlib as mpl

cmap = mpl.cm.get_cmap('coolwarm')

/tmp/ipykernel_33/1585041062.py:23: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap('coolwarm')


In [278]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasCV:", keras_cv.__version__)

TensorFlow: 2.15.0
Keras: 3.3.3
KerasCV: 0.9.0


In [279]:
class CFG:
    seed = 42
    
    # Input image size and batch size
    img_size = [128, 384]
    batch_size = 64
    
    # Audio duration, sample rate, and length
    duration = 15 # second
    sample_rate = 32000
    audio_len = duration*sample_rate
    
    # STFT parameters
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    
    # Number of epochs, model name
    epochs = 5
    preset = 'efficientnetv2_b2_imagenet'
    
    # Data augmentation parameters
    augment=True

    # Class Labels for BirdCLEF 24
    class_names = sorted(os.listdir('/kaggle/input/birdclef-2024/train_audio/'))
    num_classes = len(class_names)
    class_labels = list(range(num_classes))
    label2name = dict(zip(class_labels, class_names))
    name2label = {v:k for k,v in label2name.items()}

In [280]:
tf.keras.utils.set_random_seed(CFG.seed)

In [281]:
BASE_PATH = '/kaggle/input/birdclef-2024'


In [282]:
# df = pd.read_csv(f'{BASE_PATH}/train_metadata.csv')
# df['filepath'] = BASE_PATH + '/train_audio/' + df.filename
# df['target'] = df.primary_label.map(CFG.name2label)
# df['filename'] = df.filepath.map(lambda x: x.split('/')[-1])
# df['xc_id'] = df.filepath.map(lambda x: x.split('/')[-1].split('.')[0])

# # Display rwos
# df.head(2)

In [283]:
# def load_audio(filepath):
#     audio, sr = librosa.load(filepath)
#     return audio, sr

# def get_spectrogram(audio):
#     spec = librosa.feature.melspectrogram(y=audio, 
#                                    sr=CFG.sample_rate, 
#                                    n_mels=256,
#                                    n_fft=2048,
#                                    hop_length=512,
#                                    fmax=CFG.fmax,
#                                    fmin=CFG.fmin,
#                                    )
#     spec = librosa.power_to_db(spec, ref=1.0)
#     min_ = spec.min()
#     max_ = spec.max()
#     if max_ != min_:
#         spec = (spec - min_)/(max_ - min_)
#     return spec

# def display_audio(row):
#     # Caption for viz
#     caption = f'Id: {row.filename} | Name: {row.common_name} | Sci.Name: {row.scientific_name} | Rating: {row.rating}'
#     # Read audio file
#     audio, sr = load_audio(row.filepath)
#     # Keep fixed length audio
#     audio = audio[:CFG.audio_len]
#     # Spectrogram from audio
#     spec = get_spectrogram(audio)
#     # Display audio
#     print("# Audio:")
#     display(ipd.Audio(audio, rate=CFG.sample_rate))
#     print('# Visualization:')
#     fig, ax = plt.subplots(2, 1, figsize=(12, 2*3), sharex=True, tight_layout=True)
#     fig.suptitle(caption)
#     # Waveplot
#     lid.waveshow(audio,
#                  sr=CFG.sample_rate,
#                  ax=ax[0],
#                  color= cmap(0.1))
#     # Specplot
#     lid.specshow(spec, 
#                  sr = CFG.sample_rate, 
#                  hop_length=512,
#                  n_fft=2048,
#                  fmin=CFG.fmin,
#                  fmax=CFG.fmax,
#                  x_axis = 'time', 
#                  y_axis = 'mel',
#                  cmap = 'coolwarm',
#                  ax=ax[1])
#     ax[0].set_xlabel('');
#     fig.show()

In [284]:
# row = df.iloc[35]

# # Display audio
# display_audio(row)

In [285]:
# row = df.iloc[16]

# # Display audio
# display_audio(row)

In [286]:
# # Import required packages
# from sklearn.model_selection import train_test_split

# train_df, valid_df = train_test_split(df, test_size=0.2)

# print(f"Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")

In [287]:
# # Decodes Audio
# def build_decoder(with_labels=True, dim=1024):
#     def get_audio(filepath):
#         file_bytes = tf.io.read_file(filepath)
#         audio = tfio.audio.decode_vorbis(file_bytes)  # decode .ogg file
#         audio = tf.cast(audio, tf.float32)
#         if tf.shape(audio)[1] > 1:  # stereo -> mono
#             audio = audio[..., 0:1]
#         audio = tf.squeeze(audio, axis=-1)
#         return audio

#     def crop_or_pad(audio, target_len, pad_mode="constant"):
#         audio_len = tf.shape(audio)[0]
#         diff_len = abs(
#             target_len - audio_len
#         )  # find difference between target and audio length
#         if audio_len < target_len:  # do padding if audio length is shorter
#             pad1 = tf.random.uniform([], maxval=diff_len, dtype=tf.int32)
#             pad2 = diff_len - pad1
#             audio = tf.pad(audio, paddings=[[pad1, pad2]], mode=pad_mode)
#         elif audio_len > target_len:  # do cropping if audio length is larger
#             idx = tf.random.uniform([], maxval=diff_len, dtype=tf.int32)
#             audio = audio[idx : (idx + target_len)]
#         return tf.reshape(audio, [target_len])
    
#     def min_max_normalize(audio):
#         max_val = tf.reduce_max(audio)
#         min_val = tf.reduce_min(audio)
#         return (audio - min_val) / (max_val - min_val) * 2.0 - 1.0
    
#     def apply_preproc(spec):
#         # Standardize
#         mean = tf.math.reduce_mean(spec)
#         std = tf.math.reduce_std(spec)
#         spec = tf.where(tf.math.equal(std, 0), spec - mean, (spec - mean) / std)

#         # Normalize using Min-Max
#         max_val = tf.reduce_max(spec)
#         min_val = tf.reduce_min(spec)
#         return (spec - min_val) / (max_val - min_val) * 2.0 - 1.0

#     def get_target(target):
#         target = tf.reshape(target, [1])
#         target = tf.cast(tf.one_hot(target, CFG.num_classes), tf.float32)
#         target = tf.reshape(target, [CFG.num_classes])
#         return target
    
#     def add_noise(audio, noise_factor=0.001):
#         noise = tf.random.normal(tf.shape(audio), stddev=noise_factor)
#         return audio + noise
    
#     def trim_audio(audio, threshold=0.02):
#         # Calculate the energy of the audio signal
#         energy = tf.reduce_sum(tf.square(audio))

#         # Handle case when energy is below threshold
#         if energy < threshold:
#             return audio

#         # Find the start and end indices of the active audio segment
#         start_idx = tf.argmax(audio > threshold, output_type=tf.int32)
#         end_idx = tf.argmax(audio[::-1] > threshold, output_type=tf.int32)

#         # Trim the audio signal
#         trimmed_audio = audio[start_idx:-end_idx]
#         return trimmed_audio
    
#     def time_stretch(audio, factor_range=(0.8, 1.2)):
#         factor = tf.random.uniform([], minval=factor_range[0], maxval=factor_range[1])
#     #tf.print("Selected factor:", factor) 
#         return tfio.audio.resample(audio, rate_in=CFG.sample_rate, rate_out=tf.cast(CFG.sample_rate * factor, tf.int64))

#     def decode(path,augment= False):
#         # Load audio file
#         audio = get_audio(path)
#         # Crop or pad audio to keep a fixed length
        
# #         audio = min_max_normalize(audio)
# #         audio = trim_audio(audio)
#         if augment: 
#             if tf.random.uniform([]) < 0.35:
#                 audio = time_stretch(audio)
#             if tf.random.uniform([]) < 0.35:
#                 audio = add_noise(audio) 
#         audio = crop_or_pad(audio, dim)
#         # Audio to Spectrogram
#         spec = keras.layers.MelSpectrogram(
#             num_mel_bins=CFG.img_size[0],
#             fft_length=CFG.nfft,
#             sequence_stride=CFG.hop_length,
#             sampling_rate=CFG.sample_rate,
#         )(audio)
#         # Apply normalization and standardization
#         spec = apply_preproc(spec)
#         # Spectrogram to 3 channel image (for imagenet)
#         spec = tf.tile(spec[..., None], [1, 1, 3])
#         spec = tf.reshape(spec, [*CFG.img_size, 3])
#         return spec

#     def decode_with_labels(path, label,augment):
#         label = get_target(label)
#         return decode(path,augment), label

#     return decode_with_labels if with_labels else decode

In [288]:
# def build_augmenter():
#     augmenters = [
#         keras_cv.layers.MixUp(alpha=0.4),
#         keras_cv.layers.RandomCutout(height_factor=(1.0, 1.0),
#                                      width_factor=(0.06, 0.12)), # time-masking
#         keras_cv.layers.RandomCutout(height_factor=(0.06, 0.1),
#                                      width_factor=(1.0, 1.0)), # freq-masking
#     ]
    
#     def augment(img, label):
#         data = {"images":img, "labels":label}
#         for augmenter in augmenters:
#             if tf.random.uniform([]) < 0.35:
#                 data = augmenter(data, training=True)
#         return data["images"], data["labels"]
    
#     return augment

In [289]:
# def build_dataset(paths, labels=None, batch_size=32, 
#                   decode_fn=None, augment_fn=None, cache=True,
#                   augment=False, shuffle=2048):

#     if decode_fn is None:
#         decode_fn = build_decoder(labels is not None, dim=CFG.audio_len)

#     if augment_fn is None:
#         augment_fn = build_augmenter()
        
#     AUTO = tf.data.experimental.AUTOTUNE
#     slices = (paths,) if labels is None else (paths, labels)
#     ds = tf.data.Dataset.from_tensor_slices(slices)
#     ds = ds.map(lambda file_name, label: decode_fn(file_name, label, augment=augment), num_parallel_calls=AUTO)
#     #ds = ds.map(decode_fn, num_parallel_calls=AUTO)
#     ds = ds.cache() if cache else ds
#     if shuffle:
#         opt = tf.data.Options()
#         ds = ds.shuffle(shuffle, seed=CFG.seed)
#         opt.experimental_deterministic = False
#         ds = ds.with_options(opt)
#     ds = ds.batch(batch_size, drop_remainder=True)
#     ds = ds.map(augment_fn, num_parallel_calls=AUTO) if augment else ds
#     ds = ds.prefetch(AUTO)
#     return ds

In [290]:
# # Train
# train_paths = train_df.filepath.values
# train_labels = train_df.target.values
# train_ds = build_dataset(train_paths, train_labels, batch_size=CFG.batch_size,
#                          shuffle=True, augment=CFG.augment)

# # Valid
# valid_paths = valid_df.filepath.values
# valid_labels = valid_df.target.values
# valid_ds = build_dataset(valid_paths, valid_labels, batch_size=CFG.batch_size,
#                          shuffle=False, augment=False)

In [291]:
# def plot_batch(batch, row=3, col=3, label2name=None,):
#     """Plot one batch data"""
#     if isinstance(batch, tuple) or isinstance(batch, list):
#         specs, tars = batch
#     else:
#         specs = batch
#         tars = None
#     plt.figure(figsize=(col*5, row*3))
#     for idx in range(row*col):
#         ax = plt.subplot(row, col, idx+1)
#         lid.specshow(np.array(specs[idx, ..., 0]), 
#                      n_fft=CFG.nfft, 
#                      hop_length=CFG.hop_length, 
#                      sr=CFG.sample_rate,
#                      x_axis='time',
#                      y_axis='mel',
#                      cmap='coolwarm')
#         if tars is not None:
#             label = tars[idx].numpy().argmax()
#             name = label2name[label]
#             plt.title(name)
#     plt.tight_layout()
#     plt.show()


In [292]:
# sample_ds = train_ds.take(1)
# batch = next(iter(sample_ds))
# plot_batch(batch, label2name=CFG.label2name)

In [293]:

test_paths = glob(f'/kaggle/input/birdclef-2024/unlabeled_soundscapes/*ogg')

In [294]:
#test_paths

In [295]:
tf.keras.utils.set_random_seed(CFG.seed)


BASE_PATH = '/kaggle/input/birdclef-2024'


test_paths = glob(f'{BASE_PATH}/test_soundscapes/*ogg')
# During commit use `unlabeled` data as there is no `test` data.
# During submission `test` data will automatically be populated.
if len(test_paths)==0:
    test_paths = glob(f'{BASE_PATH}/unlabeled_soundscapes/*ogg')[:10]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df.head()

,filepath
0,/kaggle/input/birdclef-2024/unlabeled_soundsca...
1,/kaggle/input/birdclef-2024/unlabeled_soundsca...
2,/kaggle/input/birdclef-2024/unlabeled_soundsca...
3,/kaggle/input/birdclef-2024/unlabeled_soundsca...
4,/kaggle/input/birdclef-2024/unlabeled_soundsca...


In [296]:
test_paths

['/kaggle/input/birdclef-2024/unlabeled_soundscapes/1872382287.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/161691311.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/884633066.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1675425185.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1469579221.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1564606141.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/194233408.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/2003586554.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1459749597.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/33854460.ogg']

In [297]:
# Create an input layer for the model
inp = keras.layers.Input(shape=(None, None, 3))
# Pretrained backbone
backbone = keras_cv.models.EfficientNetV2Backbone.from_preset(
    CFG.preset,
)
out = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=CFG.num_classes,
    name="classifier"
)(inp)
# Build model
model = keras.models.Model(inputs=inp, outputs=out)
# Compile model with optimizer, loss and metrics
# model.compile(optimizer="adam",
#               loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
#               metrics=[keras.metrics.AUC(name='auc')],
#              )
# model.summary()



Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...


In [298]:
def check_file_existence(file_path):
    # Check if the file exists
    if os.path.exists(file_path):
        print(f"The file '{file_path}' exists.")
        return True
    else:
        print(f"The file '{file_path}' does not exist.")
        return False

# Example usage
file_path = '/kaggle/input/birdclef-24/other/fine_tune_efficient_net/1/best_model_90.weights.h5'
check_file_existence(file_path)

The file '/kaggle/input/birdclef-24/other/fine_tune_efficient_net/1/best_model_90.weights.h5' exists.


True

In [299]:
model.load_weights("/kaggle/input/birdclef-24/other/fine_tune_efficient_net/1/best_model_90.weights.h5")

In [300]:

# Decodes Audio
def build_decoder(with_labels=True, dim=1024):
    def get_audio(filepath):
        file_bytes = tf.io.read_file(filepath)
        audio = tfio.audio.decode_vorbis(file_bytes) # decode .ogg file
        audio = tf.cast(audio, tf.float32)
        if tf.shape(audio)[1]>1: # stereo -> mono
            audio = audio[...,0:1]
        audio = tf.squeeze(audio, axis=-1)
        return audio
    
    def create_frames(audio, duration=5, sr=32000):
        frame_size = int(duration * sr)
        audio = tf.pad(audio[..., None], [[0, tf.shape(audio)[0] % frame_size], [0, 0]]) # pad the end
        audio = tf.squeeze(audio) # remove extra dimension added for padding
        frames = tf.reshape(audio, [-1, frame_size]) # shape: [num_frames, frame_size]
        print(frames.shape)
        return frames
    
    def apply_preproc(spec):
        # Standardize
        mean = tf.math.reduce_mean(spec)
        std = tf.math.reduce_std(spec)
        spec = tf.where(tf.math.equal(std, 0), spec - mean, (spec - mean) / std)

        # Normalize using Min-Max
        min_val = tf.math.reduce_min(spec)
        max_val = tf.math.reduce_max(spec)
        spec = tf.where(tf.math.equal(max_val - min_val, 0), spec - min_val,
                              (spec - min_val) / (max_val - min_val))
        return spec

    def decode(path):
        # Load audio file
        audio = get_audio(path)
        # Split audio file into frames with each having 5 seecond duration
        audio = create_frames(audio)
        audio = apply_preproc(audio)
        # Convert audio to spectrogram
        spec = keras.layers.MelSpectrogram(num_mel_bins=CFG.img_size[0],
                                             fft_length=CFG.nfft, 
                                              sequence_stride=CFG.hop_length, 
                                              sampling_rate=CFG.sample_rate)(audio)
        # Apply normalization and standardization
        spec = apply_preproc(spec)
        # Covnert spectrogram to 3 channel image (for imagenet)
        spec = tf.tile(spec[..., None], [1,  1, 1, 3])
        print(spec.shape)

        return spec
     
    return decode


In [301]:
def build_dataset(paths, batch_size=1, decode_fn=None, cache=False):
    if decode_fn is None:
        decode_fn = build_decoder(dim=CFG.audio_len) # decoder
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths,)
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO) # decode audio to spectrograms then create frames
    ds = ds.cache() if cache else ds # cache files
    ds = ds.batch(batch_size, drop_remainder=False) # create batches
    ds = ds.prefetch(AUTO)
    return ds

In [302]:
# test_paths = test_df.filepath.tolist()
# test_ds = build_dataset(paths=test_paths, batch_size=CFG.batch_size)

In [303]:
# for idx, specs in enumerate(iter(test_ds)):
#     print(specs.shape)
#     specs = keras.ops.convert_to_tensor(specs[0])
#     print(specs.shape)

In [304]:
import time

ids = []

# Initialize empty array to store predictions
preds = np.empty(shape=(0, CFG.num_classes), dtype='float32')

# Build test dataset
test_paths = test_df.filepath.tolist()
test_ds = build_dataset(paths=test_paths, batch_size=1)

files_handled = 0
start_time = time.time()

# Iterate over each audio file in the test dataset
for idx, specs in enumerate(tqdm(iter(test_ds), desc='test ', total=len(test_df))):
    # Extract the filename without the extension
    filename = test_paths[idx].split('/')[-1].replace('.ogg','')
    
    # Convert to backend-specific tensor while excluding extra dimension
    specs = keras.ops.convert_to_tensor(specs[0])
    
    # Predict bird species for all frames in a recording using all trained models
    frame_preds = model.predict(specs, verbose=0)
    
    # Create a ID for each frame in a recording using the filename and frame number
    frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(frame_preds))]
    
    # Concatenate the ids
    ids += frame_ids
    # Concatenate the predictions
    preds = np.concatenate([preds, frame_preds], axis=0)
    files_handled += 1

(None, 160000)
(None, 128, 128, 3)


test : 100%|██████████| 10/10 [01:37<00:00,  9.77s/it]


In [305]:
print(f"\n{files_handled} files processed in {(time.time() - start_time)} seconds")
avg_time = ((time.time() - start_time) / files_handled)
print(f"Time / file: {avg_time} seconds")
expected_files = 1100
total_hours = (avg_time * expected_files / 3600)
print(f"Estimated time for {expected_files}: {total_hours} hours")


10 files processed in 97.93542289733887 seconds
Time / file: 9.793558692932129 seconds
Estimated time for 1100: 2.9924762672848173 hours


In [306]:
pred_df = pd.DataFrame(ids, columns=['row_id'])
pred_df.loc[:, CFG.class_names] = preds
pred_df.to_csv('/kaggle/working/submission.csv',index=False)
pred_df.head()

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,1872382287_5,0.005227,0.002690,0.000633,0.000656,0.007912,0.000712,0.001363,0.000234,0.001599,...,0.007649,0.003922,0.001839,0.002102,0.004886,0.002420,0.000276,0.000181,0.000678,0.001340
1,1872382287_10,0.002365,0.003722,0.000372,0.000627,0.008708,0.000513,0.005525,0.000165,0.002143,...,0.003044,0.001162,0.005733,0.002083,0.003058,0.003102,0.000540,0.000233,0.000574,0.001559
2,1872382287_15,0.000854,0.015027,0.000441,0.001148,0.002472,0.000176,0.020864,0.000158,0.003749,...,0.001306,0.000825,0.005603,0.012217,0.001471,0.000646,0.001246,0.000306,0.000550,0.000745
3,1872382287_20,0.003600,0.002004,0.000477,0.000346,0.013661,0.001016,0.001575,0.000323,0.002536,...,0.003176,0.002458,0.004385,0.003759,0.002755,0.005720,0.000306,0.000406,0.000671,0.003012
4,1872382287_25,0.003129,0.000452,0.000473,0.000265,0.009421,0.000679,0.000728,0.000249,0.001138,...,0.001809,0.001507,0.004953,0.001737,0.001223,0.013611,0.000200,0.000296,0.000395,0.003765


In [307]:
pred_df

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,1872382287_5,0.005227,0.002690,0.000633,0.000656,0.007912,0.000712,0.001363,0.000234,0.001599,...,0.007649,0.003922,0.001839,0.002102,0.004886,0.002420,0.000276,0.000181,0.000678,0.001340
1,1872382287_10,0.002365,0.003722,0.000372,0.000627,0.008708,0.000513,0.005525,0.000165,0.002143,...,0.003044,0.001162,0.005733,0.002083,0.003058,0.003102,0.000540,0.000233,0.000574,0.001559
2,1872382287_15,0.000854,0.015027,0.000441,0.001148,0.002472,0.000176,0.020864,0.000158,0.003749,...,0.001306,0.000825,0.005603,0.012217,0.001471,0.000646,0.001246,0.000306,0.000550,0.000745
3,1872382287_20,0.003600,0.002004,0.000477,0.000346,0.013661,0.001016,0.001575,0.000323,0.002536,...,0.003176,0.002458,0.004385,0.003759,0.002755,0.005720,0.000306,0.000406,0.000671,0.003012
4,1872382287_25,0.003129,0.000452,0.000473,0.000265,0.009421,0.000679,0.000728,0.000249,0.001138,...,0.001809,0.001507,0.004953,0.001737,0.001223,0.013611,0.000200,0.000296,0.000395,0.003765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,33854460_220,0.004145,0.002037,0.001667,0.001131,0.017875,0.000818,0.001224,0.005695,0.001042,...,0.004408,0.073656,0.004788,0.006899,0.003349,0.012494,0.000452,0.015104,0.000547,0.001968
476,33854460_225,0.004877,0.001318,0.001179,0.000816,0.008272,0.000514,0.000591,0.002486,0.000533,...,0.011763,0.210128,0.001771,0.003244,0.003298,0.002900,0.000355,0.004016,0.000707,0.001792
477,33854460_230,0.002422,0.003406,0.002142,0.001031,0.027336,0.000666,0.001494,0.005899,0.000683,...,0.009321,0.037942,0.003056,0.007153,0.005548,0.012552,0.000496,0.009856,0.000867,0.002272
478,33854460_235,0.002180,0.003785,0.001617,0.000688,0.034679,0.000545,0.000799,0.004359,0.000840,...,0.008000,0.033929,0.003315,0.003959,0.018962,0.022027,0.000374,0.008114,0.000796,0.001891


In [308]:
# import math

# def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
#     lr_start, lr_max, lr_min = 5e-5, 8e-6 * batch_size, 1e-5
#     lr_ramp_ep, lr_sus_ep, lr_decay = 3, 0, 0.75

#     def lrfn(epoch):  # Learning rate update function
#         if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
#         elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
#         elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
#         elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
#         elif mode == 'cos':
#             decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
#             phase = math.pi * decay_epoch_index / decay_total_epochs
#             lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
#         return lr

#     if plot:  # Plot lr curve if plot is True
#         plt.figure(figsize=(10, 5))
#         plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
#         plt.xlabel('epoch'); plt.ylabel('lr')
#         plt.title('LR Scheduler')
#         plt.show()

#     return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

In [309]:
# lr_cb = get_lr_callback(CFG.batch_size, plot=True)

In [310]:
# ckpt_cb = keras.callbacks.ModelCheckpoint("best_model.weights.h5",
#                                          monitor='val_auc',
#                                          save_best_only=True,
#                                          save_weights_only=True,
#                                          mode='max')

In [311]:
# history = model.fit(
#     train_ds, 
#     validation_data=valid_ds, 
#     epochs=CFG.epochs,
#     callbacks=[lr_cb, ckpt_cb], 
#     verbose=1
# )

In [312]:
# train_df = pd.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv')
# relation_df = pd.read_csv('/kaggle/input/birdclef-2024/eBird_Taxonomy_v2021.csv')

In [313]:
# train_df.tail()

### Load and decode audio data

# Detect Number of channel in audio file

In [314]:
# import tensorflow as tf
# import tensorflow_io as tfio

# # Function to load an OGG file
# def load_ogg_audio(filename):
#     # Read the file
#     # Decode the OGG file
#     audio_data = tfio.audio.AudioIOTensor(filename)
#     return audio_data

# # Replace 'your_audio_file.ogg' with your actual OGG audio file path
# audio = load_ogg_audio('/kaggle/input/birdclef-2024/train_audio/asbfly/XC164848.ogg')

# # Convert the audio tensor to a numpy array


### Inferencing

In [315]:
# audio

### Spectogram

In [316]:
# audio_slice = audio[:96000]

# # No need to squeeze
# audio_tensor = tf.squeeze(audio_slice, axis=[-1])

In [317]:
# audio_tensor

In [318]:
# from IPython.display import Audio

# Audio(audio_tensor.numpy(), rate=audio.rate.numpy())

In [319]:
# import matplotlib.pyplot as plt


# tensor = tf.cast(audio_tensor, tf.float32) / 32768.0

# plt.figure()
# plt.plot(tensor.numpy())

In [320]:
# tensor

In [321]:
# min_value = tf.reduce_min(tensor)

# # Get the maximum value
# max_value = tf.reduce_max(tensor)

# print("Minimum value:", min_value.numpy())
# print("Maximum value:", max_value.numpy())

In [322]:
# position = tfio.audio.trim(tensor, axis=0, epsilon=0.0000004)
# print(position)

# start = position[0]
# stop = position[1]
# print(start, stop)

# processed = tensor[start:stop]

# plt.figure()
# plt.plot(processed.numpy())

# Fading the audio signal 

In [323]:
# fade = tfio.audio.fade(
#     processed, fade_in=1000, fade_out=1000, mode="logarithmic")

# plt.figure()
# plt.plot(fade.numpy())

# Spectogram Visualization

In [324]:
# # Convert to spectrogram
# spectrogram = tfio.audio.spectrogram(
#     fade, nfft=1024, window=1024, stride=128)

# plt.figure()
# plt.imshow(tf.math.log(spectrogram).numpy())

In [325]:
# # Convert to mel-spectrogram
# mel_spectrogram = tfio.audio.melscale(
#     spectrogram, rate=18000, mels=128, fmin=0, fmax=8000)


# plt.figure()
# plt.imshow(tf.math.log(mel_spectrogram).numpy())

# # Convert to db scale mel-spectrogram
# dbscale_mel_spectrogram = tfio.audio.dbscale(
#     mel_spectrogram, top_db=80)

# plt.figure()
# plt.imshow(dbscale_mel_spectrogram.numpy())

In [326]:
#spectrogram = tfio.audio.spectrogram(audio_tensor, nfft=2048, window=2048, stride=512)

# Frequency Distribution of each  birds category

In [327]:
# import matplotlib.pyplot as plt
# import seaborn as sbn

In [328]:
# category_counts = train_df['primary_label'].value_counts()

In [329]:
# category_counts

In [330]:


# # Plotting the horizontal bar chart
# plt.figure(figsize=(15,20 ))
# category_counts.plot(kind='barh', color='skyblue')
# plt.xlabel('Count')
# plt.ylabel('Category')
# plt.title('Distribution of Categories')
# plt.grid(axis='x', linestyle='--', alpha=0.7)
# plt.show()

In [331]:
# len(train_df['primary_label'].unique())

In [332]:
# relation_df.head(5)

In [333]:
# relation_df['CATEGORY'].unique()

In [334]:
# relation_df['SPECIES_CODE'].unique()

In [335]:
# relation_df.info()

# Configuration

In [336]:
# import os
# class CFG:
#     seed = 42
    
#     # Input image size and batch size
#     img_size = [128, 384]
#     batch_size = 32
    
#     # Audio duration, sample rate, and length
#     duration = 5 # second
#     sample_rate = 32000
#     audio_len = duration*sample_rate
    
#     # STFT parameters
#     nfft = 2028
#     window = 2048
#     hop_length = audio_len // (img_size[1] - 1)
#     fmin = 20
#     fmax = 16000
    
#     # Number of epochs, model name
#     epochs = 5
#     preset = 'efficientnetv2_b2_imagenet'
    
#     # Data augmentation parameters
#     augment=True

#     # Class Labels for BirdCLEF 24
#     class_names = sorted(os.listdir('/kaggle/input/birdclef-2024/train_audio/'))
#     num_classes = len(class_names)
#     class_labels = list(range(num_classes))
#     label2name = dict(zip(class_labels, class_names))
#     name2label = {v:k for k,v in label2name.items()}

In [337]:
# BASE_PATH = '/kaggle/input/birdclef-2024'

In [338]:
# train_df['file_path'] = BASE_PATH + '/train_audio/' + train_df.filename
# train_df['target']  = train_df.primary_label.map(CFG.name2label)

In [339]:
# train_df.columns

In [340]:
# import os
# os.environ["KERAS_BACKEND"] = "tensorflow"
# import tensorflow as tf
# import tensorflow_io as tfio
#   # "jax" or "tensorflow" or "torch" 

# import keras_cv
# import keras



# Preprocessing

In [341]:
# file_path_tensor = tf.constant(train_df['file_path'].values)
# target_tensor = tf.constant(train_df['target'].values)

In [342]:
#dataset = tf.data.Dataset.from_tensor_slices((file_path_tensor,target_tensor))

In [343]:
# dataset = dataset.shuffle(1024, seed=123)
# dataset = dataset.batch(32, drop_remainder=True);
# dataset  = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [344]:
# sample_ds = dataset.take(2)
# it = iter(sample_ds)
# print(next(it))

In [345]:
# @tf.function
# def min_max_normalize(audio):
#     max_val = tf.reduce_max(audio)
#     min_val = tf.reduce_min(audio)
#     return (audio - min_val) / (max_val - min_val) * 2.0 - 1.0

In [346]:
# def calculate_energy(audio, window_size=0.02):
#     # Calculate energy of audio signal using sliding window approach
#     audio = tf.expand_dims(audio, axis=0)
#     window_length = int(window_size * audio.shape[0])
#     # Square the audio signal
#     squared_audio = tf.square(audio)
#     # Calculate the energy of the audio signal using a sliding window approach
#     energy = tf.nn.conv1d(squared_audio, tf.ones((window_length, 1, 1)), stride=1, padding='VALID')
#     return energy

# def determine_threshold(energy, percentile=90):
#     # Determine threshold based on energy distribution
#     threshold = tfp.stats.percentile(energy, percentile)
#     return threshold


In [347]:
# @tf.function
# def trim_audio(audio, threshold=0.1):
#        # Calculate the energy of the audio signal
#     energy = tf.reduce_sum(tf.square(audio), axis=-1)

#     # Find the start and end indices of the active audio segments
#     start_idx = tf.argmax(energy > threshold)
#     end_idx = tf.argmax(energy[::-1] > threshold)

#     # Trim the audio signal
#     trimmed_audio = audio[start_idx:-end_idx]
#     #print(trimmed_audio)
#     return trimmed_audio

In [348]:
# @tf.function
# def crop_or_pad(audio, target_len, pad_mode="constant"):
#     audio = tf.reshape(audio, [-1])  # Ensure audio is a 1-D tensor
#     audio_len = tf.shape(audio)[0]
    
#     diff_len = tf.abs(target_len - audio_len)  # Find difference between target and audio length
    
#     if audio_len < target_len:  # Do padding if audio length is shorter
#         pad1 = tf.random.uniform([], maxval=diff_len, dtype=tf.int32)
#         pad2 = diff_len - pad1
#         audio = tf.pad(audio, paddings=[[pad1, pad2]], mode=pad_mode)
#     elif audio_len > target_len:  # Do cropping if audio length is larger
#         idx = tf.random.uniform([], maxval=diff_len, dtype=tf.int32)
#         audio = audio[idx : (idx + target_len)]
    
#     return tf.reshape(audio, [target_len])

In [349]:
# @tf.function
# def apply_preproc(spec):
#         # Standardize
#         mean = tf.math.reduce_mean(spec)
#         std = tf.math.reduce_std(spec)
#         spec = tf.where(tf.math.equal(std, 0), spec - mean, (spec - mean) / std)
        
#         return min_max_normalize(spec)
       

In [350]:
# import matplotlib.pyplot as plt


In [351]:
# @tf.function
# def time_stretch(audio, factor_range=(0.8, 1.2)):
#     factor = tf.random.uniform([], minval=factor_range[0], maxval=factor_range[1])
#     #tf.print("Selected factor:", factor) 
#     return tfio.audio.resample(audio, rate_in=CFG.sample_rate, rate_out=tf.cast(CFG.sample_rate * factor, tf.int64))

In [352]:
# @tf.function
# def get_target(target):
#         target = tf.reshape(target, [1])
#         target = tf.cast(tf.one_hot(target, CFG.num_classes), tf.float32)
#         target = tf.reshape(target, [CFG.num_classes])
#         return target

In [353]:
# mel_layer = tf.keras.layers.MelSpectrogram(
#     num_mel_bins=CFG.img_size[0],
#     fft_length=CFG.nfft,
#     sequence_stride=CFG.hop_length,
#     sampling_rate=CFG.sample_rate,
# )

# @tf.function
# def pre_processing(file_name,augment):
#     #tf.print("Start Preprocessing") 
#     audio_data = tfio.audio.AudioIOTensor(file_name,dtype=tf.float32)
#     audio_tensor = audio_data.to_tensor()
#     #audio_tensor = tf.squeeze(audio_tensor, axis=[-1])
#     #audio_tensor = min_max_normalize(audio_tensor)
#     #audio_tensor = trim_audio(audio_tensor)
# #     if augment:
# #         if tf.random.uniform([]) < 0.35:
# #             audio_tensor = time_stretch(audio_tensor)
#     audio_tensor = crop_or_pad(audio_tensor,CFG.audio_len)
#     audio_tensor =  tf.keras.layers.MelSpectrogram(
#             num_mel_bins=CFG.img_size[0],
#             fft_length=CFG.nfft,
#             sequence_stride=CFG.hop_length,
#             sampling_rate=CFG.sample_rate,
#         )(audio_tensor)
#     audio_tensor = apply_preproc(audio_tensor)
#     #tf.print("End Preprocessing") 
#     return audio_tensor
    

In [354]:
# @tf.function 
# def decode_with_labels(path, label,augment):
#         #tf.print("Start Deconding") 
#         label = get_target(label)
#         return pre_processing(path,augment), label

In [355]:
# @tf.function
# def add_noise(audio, noise_factor=0.001):
#     noise = tf.random.normal(tf.shape(audio), stddev=noise_factor)
#     return audio + noise

# @tf.function
# def augment(data, label):
# #     if tf.random.uniform([]) < 0.35:
# #         data = add_noise(data)
    
#     data = tf.tile(data[..., None], [1, 1, 3])
#     data = tf.reshape(data, [*CFG.img_size, 3])
    
# #     time_masking = keras_cv.layers.RandomCutout(height_factor=(1.0, 1.0),
# #                                      width_factor=(0.06, 0.12))
# #     frequency_masking = keras_cv.layers.RandomCutout(height_factor=(0.06, 0.1),
# #                                      width_factor=(1.0, 1.0))
    
# #     if tf.random.uniform([]) < 0.35:
# #             data = time_masking(data)
    
# #     if tf.random.uniform([]) < 0.35:
# #             data = frequency_masking(data)
    
#     return data, label


In [356]:
# @tf.function
# def build_dataset(paths, labels=None, batch_size=16, 
#                    cache=True, shuffle=128,augment1=False):

#     AUTO = tf.data.experimental.AUTOTUNE
#     slices = (paths,) if labels is None else (paths, labels)
#     ds = tf.data.Dataset.from_tensor_slices(slices)
#     ds = ds.cache() if cache else ds
#     if shuffle:
#         opt = tf.data.Options()
#         ds = ds.shuffle(shuffle, seed=CFG.seed)
#         opt.experimental_deterministic = False
#         ds = ds.with_options(opt)
#     ds = ds.map(lambda file_name, label: decode_with_labels(file_name, label, augment=augment1), num_parallel_calls=AUTO)
#     ds = ds.map(augment, num_parallel_calls=AUTO) if augment1 else ds
#     ds = ds.batch(batch_size, drop_remainder=True)
#     ds = ds.prefetch(AUTO)
#     return ds

In [357]:
# a_t, rate = decode_with_labels('/kaggle/input/birdclef-2024/train_audio/asbfly/XC309402.ogg',1,True)

In [358]:
# at,rate = augment(a_t,rate)

In [359]:
# at

In [360]:
# # Import required packages
# from sklearn.model_selection import train_test_split


# tn_df, valid_df = train_test_split(train_df, test_size=0.02)

# print(f"Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")

In [361]:
# valid_df

# Dataset Testing

In [362]:
# train_paths = tf.constant(valid_df.file_path.values)
# train_labels = tf.constant(valid_df.target.values)

In [363]:
# train_labels

In [364]:
# AUTO = tf.data.experimental.AUTOTUNE

In [365]:
#boolean_tensor = tf.constant([True] * len(train_paths))

In [366]:
# train_paths = tf.constant(valid_df.file_path.values)
# train_labels = tf.constant(valid_df.target.values)

In [367]:
# ds = tf.data.Dataset.from_tensor_slices((train_paths,train_labels))


In [368]:
# ds = ds.shuffle(30, seed=CFG.seed)

In [369]:
# ds = ds.map(lambda file_name, label: decode_with_labels(file_name, label, augment=True), num_parallel_calls=AUTO)

In [370]:
# ds = ds.map(augment, num_parallel_calls=AUTO)

In [371]:
# ds = ds.batch(32, drop_remainder=True)

In [372]:

#
# ds = ds.prefetch(AUTO)

In [373]:
#ds

In [374]:

#next(de)

In [375]:
# for batch in valid_ds.take(1):
#     images, labels = batch
#     print(f'Images shape: {images.shape}')
#     print(f'Labels shape: {labels.shape}')

In [376]:
#sd = ds.take(1)

In [377]:
# CFG.augment = True
# train_paths = tf.constant(tn_df.file_path.values)
# train_labels = tf.constant(tn_df.target.values)
# train_ds = build_dataset(train_paths, train_labels,augment1=True,batch_size=CFG.batch_size)

# # Valid
# CFG.augment = False
# valid_paths = tf.constant(valid_df.file_path.values)
# valid_labels = tf.constant(valid_df.target.values)
# valid_ds = build_dataset(valid_paths, valid_labels,batch_size = CFG.batch_size,augment1=True)

# Traning

In [378]:
# from tensorflow.keras.applications import EfficientNetV2B2
# from tensorflow.keras.layers import Concatenate, Dense, GlobalAveragePooling2D
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input

In [379]:
# inp = keras.layers.Input(shape=(None, None, 3))
# base_model = EfficientNetV2B2(include_top=False, weights='imagenet', input_shape=(None, None, 3))
# base_model.trainable = False

# x = base_model(inp)
# x = GlobalAveragePooling2D()(x)
# x= keras.layers.Dense(CFG.num_classes, activation='softmax')(x)

# model = Model(inputs=inp , outputs=x)

# # Compile the model
# model.compile(optimizer="adam",
#              loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
#               metrics=[keras.metrics.AUC(name='auc')],
#              )
# # Print the model summary
# model.summary()

In [380]:
# # Create an input layer for the model
# inp = keras.layers.Input(shape=(None, None, 3))
# # Pretrained backbone
# backbone = keras_cv.models.EfficientNetV2Backbone.from_preset(
#     CFG.preset,
# )
# backbone.layers
# out = keras_cv.models.ImageClassifier(
#     backbone=backbone,
#     num_classes=CFG.num_classes,
#     name="classifier"
# )(inp)
# # Build model
# model = keras.models.Model(inputs=inp, outputs=out)

# model.layers[-1].trainable = False
    
# # Compile model with optimizer, loss and metrics
# model.compile(optimizer="adam",
#               loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
#               metrics=[keras.metrics.AUC(name='auc')],
#              )

# print("Last three layers of the model:")
# # for layer in model.layers[-1]:
# #     print(f"Layer name: {layer.name}, Layer type: {type(layer)}, Trainable: {layer.trainable}")

# model.summary()

In [381]:
import matplotlib.pyplot as plt

In [382]:
# import math

# def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
#     lr_start, lr_max, lr_min = 5e-5, 8e-6 * batch_size, 1e-5
#     lr_ramp_ep, lr_sus_ep, lr_decay = 3, 0, 0.75

#     def lrfn(epoch):  # Learning rate update function
#         if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
#         elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
#         elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
#         elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
#         elif mode == 'cos':
#             decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
#             phase = math.pi * decay_epoch_index / decay_total_epochs
#             lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
#         return lr

#     if plot:  # Plot lr curve if plot is True
#         plt.figure(figsize=(10, 5))
#         plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
#         plt.xlabel('epoch'); plt.ylabel('lr')
#         plt.title('LR Scheduler')
#         plt.show()

#     return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

In [383]:
# lr_cb = get_lr_callback(CFG.batch_size, plot=True)

In [384]:
ckpt_cb = keras.callbacks.ModelCheckpoint("best_model.weights.h5",
                                         monitor='val_auc',
                                         save_weights_only=True,
                                         mode='max')

In [385]:
# valid_ds

In [386]:
# if gpus:
#     history = model.fit(
#         valid_ds, 
#         epochs=CFG.epochs,
#         verbose=1
#     )

In [387]:
# model.predict(valid_ds)

In [388]:
# import librosa.display as lid

In [389]:
# def plot_batch(batch, row=3, col=3, label2name=None,):
#     """Plot one batch data"""
#     if isinstance(batch, tuple) or isinstance(batch, list):
#         specs, tars = batch
#     else:
#         specs = batch
#         tars = None
#     plt.figure(figsize=(col*5, row*3))
#     for idx in range(row*col):
#         ax = plt.subplot(row, col, idx+1)
#         lid.specshow(np.array(specs[idx, ..., 0]), 
#                      n_fft=CFG.nfft, 
#                      hop_length=CFG.hop_length, 
#                      sr=CFG.sample_rate,
#                      x_axis='time',
#                      y_axis='mel',
#                      cmap='coolwarm')
#         if tars is not None:
#             label = tars[idx].numpy().argmax()
#             name = label2name[label]
#             plt.title(name)
#     plt.tight_layout()
#     plt.show()

In [390]:
# CFG.augment = False

In [391]:
#sample_ds = train_ds.take(1)

In [392]:
#next(iter(sample_ds))

In [393]:
# s = valid_ds.take(1)
# de = iter(s)
# sr = next(de)
# plot_batch(sr, label2name=CFG.label2name)

In [394]:
# plt.figure()
# plt.plot(ag.numpy())

In [395]:
# trimmed_audio  = trim_audio(a_t)

In [396]:
# plt.figure()
# plt.plot(trimmed_audio.numpy())

In [397]:
# a_t.shape

In [398]:
# a_t = tf.squeeze(trimmed_audio, axis=[-1])

In [399]:
# from IPython.display import Audio

# Audio(a_t.numpy(), rate=rate.numpy())

## Remove background noise

In [400]:
# !pip install pywavelets

In [401]:
# import pywt

# def wavelet_denoising(audio_signal, wavelet='db4', level=1, threshold_mode='soft', threshold_value=0.1):
#     # Decompose audio signal into wavelet coefficients
#     coeffs = pywt.wavedec(audio_signal.numpy(), wavelet, level=level)
    
#     # Apply thresholding to each detail coefficient
#     for i in range(1, len(coeffs)):
#         coeffs[i] = pywt.threshold(coeffs[i], threshold_value, mode=threshold_mode)
    
#     # Reconstruct denoised signal from thresholded coefficients
#     denoised_signal = pywt.waverec(coeffs, wavelet)
    
#     return tf.convert_to_tensor(denoised_signal, dtype=tf.float32)

In [402]:
# denoised_signal = wavelet_denoising(a_t,level=4)

In [403]:
# plt.figure()
# plt.plot(trimmed_audio.numpy())

In [404]:
# from IPython.display import Audio

# Audio(denoised_signal.numpy(), rate=rate.numpy())

In [405]:
# plt.figure()
# plt.plot(denoised_signal.numpy())

In [406]:
# plt.figure()
# plt.plot(trimmed_audio.numpy())